In [ ]:
# Set global utool flags
import utool as ut
ut.util_io.__PRINT_WRITES__ = False
ut.util_io.__PRINT_READS__ = False
ut.util_parallel.__FORCE_SERIAL__ = True
ut.util_cache.VERBOSE_CACHE = False
ut.NOT_QUIET = False

# Matplotlib stuff
%pylab inline
%load_ext autoreload
%autoreload

# Define database for this test run
import wbia
#db = 'PZ_Master1'
#db = 'seals2'
#db = 'GZ_ALL'
#db = 'NNP_MasterGIRM_core'
#db = 'turtles'
#db = 'WS_Hard'
db = 'WS_ALL'

# Setup database specific configs
ctrl = 'ctrl'
unctrl = 'unctrl'
a = ['ctrl']

if db == 'PZ_MTEST' or db == 'GZ_ALL':
    a = ['ctrl']
elif db == 'PZ_Master1':
    a = ['timectrl']
elif db == 'WS_HARD':
    pass
elif db == 'WS_ALL':
    ctrl = 'ctrl:been_adjusted=True'
    a = [ctrl]

if db == 'PZ_Master1':
    varysize = ['varysize_pzm']
    varypername = ['varypername_pzm']
elif db == 'GZ_ALL':
    varysize = ['varysize_gz']
    varypername = ['varypername_gz']
elif db == 'NNP_MasterGIRM_core':
    varysize = ['varysize_girm']
    varypername = ['varypername_girm']

# Database Information

In [ ]:
import wbia
ibs = wbia.opendb(db=db)
wbia.other.dbinfo.show_image_time_distributions(ibs, ibs.get_valid_gids())

In [ ]:
_ = wbia.other.dbinfo.get_dbinfo(ibs)

# Detection Summary

In [ ]:
# Get a sample of images
gids = ibs.get_valid_gids()
aids = ibs.get_image_aids(gids)
nAids_list = list(map(len, aids))
gids_sorted = ut.sortedby(gids, nAids_list)[::-1]
samplex = list(range(5))
print(samplex)
gids_sample = ut.list_take(gids_sorted, samplex)

import wbia.viz
for gid in ut.ProgressIter(gids_sample, lbl='drawing image'):
    wbia.viz.show_image(ibs, gid)

# Identification Summary

### Identification Accuracy
The uncontrolled
The baseline pipeline configuration is: 

In [ ]:
a = ['unctrl', ctrl]
if db in ['WS_ALL', 'WS_Hard']:
    a = [ctrl]

test_result = wbia.run_experiment(
    e='rank_cdf', 
    db=db, 
    a=a, 
    t=['baseline'])
#test_result.print_unique_annot_config_stats()
_ = test_result.draw_func()

### Distribution of Correct Matches (True Positives) over timedelta categories

In [ ]:
test_result = wbia.run_experiment(
    e='timedelta_hist',
    db=db, 
    a=a,
    t=['baseline'],
    truepos=True)
test_result.draw_func()

### Distribution of Incorrect Matches (False Positives) over timedelta categories

In [ ]:
# Baseline Timedeltas FP
test_result = wbia.run_experiment(
    e='timedelta_hist',
    db=db, 
    a=a,
    t=['baseline'],
    falsepos=True)
test_result.draw_func()

# Identification using different pipeline configuration
This test varies the feature invariance used in the identification algorithm


In [ ]:
test_result = wbia.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['ctrl'], 
    t=['invar4'])
test_result.draw_func()

# Score Separability Experiments
These plots show the distribution of identification scores for groundtrue (correct) and groundfalse (incorrect) matches

In [ ]:
a = a
t = ['best']

## Scores of Success Cases

In [ ]:
test_result = wbia.run_experiment(
    e='scores',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=False,min_gf_timedelta=None'],
)
_ = test_result.draw_func()

## Scores of All Cases

In [ ]:
# Score Seperability with NO FILTERING
test_result = wbia.run_experiment(
    e='scores',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=None,min_gf_timedelta=None']
)
_ = test_result.draw_func()

------------------

# Individual Results 
* Higher scores are on the Left
* Groundtrue matches have a green border

In [ ]:
t = ['best']
a = a
draw_case_kw = dict(show_in_notebook=True, annot_modes=[0, 1])

### Highest Scoring Success Cases

In [ ]:
test_result = wbia.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=False,min_gf_timedelta=12h,index=0:3,sortdsc=gtscore,without_gf_tag=Photobomb'],
    **draw_case_kw)
_ = test_result.draw_func()

### Failure Cases (sorted by groundtrue score)

In [ ]:
test_result = wbia.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=True,min_gf_timedelta=12h,index=0:3,sortdsc=gtscore'],
    **draw_case_kw)
_ = test_result.draw_func()

### Failure Cases (sorted by groundfalse score) 

In [ ]:
test_result = wbia.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=True,min_gf_timedelta=12h,index=0:3,sortdsc=gfscore'],
    **draw_case_kw)
_ = test_result.draw_func()